In [2]:
import pandas as pd 
import os
from datetime import datetime, time,datetime
from fyers_apiv3 import fyersModel
from StockFetcher import get_long_short_universe
import pandas as pd
import yfinance as yf
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
def calculate_difference_metric(rolling_values_df, target_date, trade_type='long'):
    target = rolling_values_df.loc[rolling_values_df.index.date < target_date].tail(1).T
    target.rename(columns={target.columns[0]: 'LinearReg'}, inplace=True)
   
    if trade_type == 'long':
        target = target[target['LinearReg'] > 0]
    elif trade_type == 'short':
        target = target[target['LinearReg'] < 0]
    ascending_order = (trade_type == 'short')
    sorted_columns = target.sort_values('LinearReg', ascending=ascending_order).index
    
    return sorted_columns

def get_long_short_universe(stocks, stocks_to_select=2):
    MasterDf = yf.download(tickers=stocks, period='7d')[["Close", "Open"]]
    MasterDf = MasterDf.dropna(axis=1)
    MasterDf = MasterDf.loc[:, (MasterDf <= 7500).all()]
    MasterDf.index = pd.to_datetime(MasterDf.index)
    CloseDf = MasterDf['Close'][:-1]
    OpenDf = MasterDf['Open']
    rolling_values_df = (CloseDf - CloseDf.rolling(5).mean()) / CloseDf
    
    long_universe = calculate_difference_metric(rolling_values_df, CloseDf.index[-1], 'long')
    short_universe = calculate_difference_metric(rolling_values_df, CloseDf.index[-1], 'short')
    CloseDf_long = MasterDf['Close'][long_universe][:-1]
    OpenDf_long = MasterDf['Open'][long_universe]

    CloseDf_short = MasterDf['Close'][short_universe][:-1]
    OpenDf_short = MasterDf['Open'][short_universe]
    
    long_stocks = final_stocks(OpenDf_long, CloseDf_long, OpenDf_long.index[-1], CloseDf_long.index[-1], trade_type='long', lst=long_universe, stocks=stocks_to_select)
    short_stocks = final_stocks(OpenDf_short, CloseDf_short, OpenDf_short.index[-1], CloseDf_short.index[-1], trade_type='short', lst=short_universe, stocks=stocks_to_select)
    
    long_stocks = [f'NSE:{stock[:-3]}-EQ' for stock in long_stocks]
    short_stocks = [f'NSE:{stock[:-3]}-EQ' for stock in short_stocks]


    return long_stocks, short_stocks

def final_stocks(Open_df, Close_df, present_date, past_date, trade_type='long', lst=None,stocks=2):
    if present_date < past_date:
        print('False')
        
    open_prices_present = Open_df.loc[Open_df.index.date == present_date].head(1).T
    close_prices_past = Close_df.loc[Close_df.index.date == past_date].tail(1).T
    gap = (open_prices_present.values - close_prices_past.values) / close_prices_past.values
    
    result_df = pd.DataFrame(gap, columns=['Gap'])
    result_df['Stocks'] = open_prices_present.index
    result_df.set_index('Stocks', inplace=True)
    result_df.dropna(inplace=True)
    
    ascending_order = (trade_type.lower() == 'long')
    
    result_df.sort_values('Gap', ascending=ascending_order, inplace=True)
    
    if lst is not None:
        intersection = result_df.index.to_list()
        return intersection[0:stocks]
    else:
        return []


In [8]:
from Autologin import generateToken
client_id = "CDAAGPIZ8E-100"


access_token=generateToken()
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")
fyers.orderbook()

send_login_otp success
generate_totp success
614533
verify_totp_result success
verify_pin_result success
token_result success
validate_authcode success
access_token - eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MDczMTYxMjYsImV4cCI6MTcwNzM1MjI0NiwibmJmIjoxNzA3MzE2MTI2LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbHc1T2VqWkVqUFRCTmlqQW1NTHBJYk9DSEVTRWRZai1rek04bXQ2eVhVbnNYcjlycVl4RE1vTndUVEJ4a1hUSE5ELXFfcHh3M00wTGQyamVJcXJKSTVibFZRMXlUOVIyWkF3M250YkRSUXNjeHB6QT0iLCJkaXNwbGF5X25hbWUiOiJTT0hVTSBNVUxFWSIsIm9tcyI6IksxIiwiaHNtX2tleSI6ImYxYzA3NWQ0Y2Q2ZmQ4MjM0ZTM4MDdmNWI2OTEzYTMzOGRiNDQ1ODU1ZGJhMWZmM2Q0YTc1NTIyIiwiZnlfaWQiOiJYUzU1MDgzIiwiYXBwVHlwZSI6MTAwLCJwb2FfZmxhZyI6Ik4ifQ.NLXEX-snEMaS9GyVhUJnRM4eeNFcoSrqUSN5e2olcc4


{'code': 200, 'message': '', 's': 'ok', 'orderBook': []}

In [9]:
fyers.positions()

{'code': 200,
 'message': '',
 's': 'ok',
 'netPositions': [],
 'overall': {'count_open': 0,
  'count_total': 0,
  'pl_realized': 0,
  'pl_total': 0,
  'pl_unrealized': 0}}

In [10]:
from datetime import datetime, time,datetime
import time as sleep_time
from IPython.display import clear_output

from datetime import datetime
def date_to_epoch():
    current_date_time = datetime.now()
    epoch_time_current = int(current_date_time.timestamp())
    return epoch_time_current


def create_order(symbol, side,capital):

    data = {
        "symbol":symbol,
        "resolution":"D",
        "date_format":"0",
        "range_from":date_to_epoch(),
        "range_to":date_to_epoch(),
        "cont_flag":"1"
    }
    response = fyers.history(data=data)
    openprice=float(pd.DataFrame(response['candles'],columns=['Date','Open','High','Low','Close','Volume'])['Open'])
    quantity= int(capital/openprice)
    order_dict = {
        "symbol": symbol,
        "qty": quantity,
        "type": 2,
        "side": side,
        "productType": "INTRADAY",
        "limitPrice": 0,
        "stopPrice": 0,
        "validity": "DAY",
        "disclosedQty": 0,
        "offlineOrder": False,
    }
    return order_dict

target_time_long = time(9, 26)
target_time_short = time(9, 15, 0, 1)  
target_time_exit = time(15, 14)
stocks_list = pd.read_csv('ind_nifty500list.csv')['Symbol'].to_list()

long_orders=[]
short_orders=[]


long_stocks,short_stocks=get_long_short_universe(stocks_list,2)
Master_capital=5* int(fyers.funds()['fund_limit'][8]['equityAmount'])
partial_capital=Master_capital/(2*len(long_stocks))


for  i in range(len(long_stocks)):
    long_orders.append(create_order(long_stocks[i],1,partial_capital))

for  i in range(len(short_stocks)):
    short_orders.append(create_order(short_stocks[i],-1,partial_capital))

long_executed = False
short_executed = False
exit_executed = False

while True:
    current_time = datetime.now().time()

    if not long_executed and current_time >= target_time_long:
        
              
        response_long = fyers.place_basket_orders(data=long_orders)
        print("Long order executed.")
        long_executed = True

    elif not short_executed and current_time >= target_time_short:
        response_short = fyers.place_basket_orders(data=short_orders)
        print("Short order executed.")
        short_executed = True

    elif not exit_executed and current_time >= target_time_exit:
        # Close all positions at exit time
        response_exit = fyers.exit_positions()
        print("Exiting all positions.")
        exit_executed = True
       
        df = pd.DataFrame(fyers.orderbook()['orderBook'])
        df = df[['id', 'qty', 'symbol', 'orderDateTime', 'side', 'tradedPrice', 'orderNumStatus', 'slNo']]

        # Your project folder path
        project_folder = r"C:\Users\Sohum\Desktop\Long Short Deployment"

        # CSV file path
        csv_file_path = os.path.join(project_folder, "tradebook.csv")

        # Append the DataFrame to the CSV file
        df.to_csv(csv_file_path, mode='a', header=not os.path.exists(csv_file_path), index=False)

        print("DataFrame appended to CSV:", csv_file_path)
    elif long_executed and short_executed and exit_executed:
        # Break out of the loop once all conditions are executed
        break

    else:
        print(f"Waiting for entry or exit time. Current time: {current_time}")
        clear_output(wait=True)



    sleep_time.sleep(1)

Exiting all positions.
DataFrame appended to CSV: C:\Users\Sohum\Desktop\Long Short Deployment\tradebook.csv


In [1]:
import yfinance as yf
import pandas as pd
from StockFetcher import get_long_short_universe

stocks_list = pd.read_csv('ind_nifty500list.csv')['Symbol'].to_list()

long_stocks,short_stocks=get_long_short_universe(stocks_list,4)
long_stocks=[stock.replace('NSE:', '').replace('-EQ', '.NS') for stock in long_stocks]
short_stocks=[stock.replace('NSE:', '').replace('-EQ', '.NS') for stock in short_stocks]
print(long_stocks,short_stocks)
long=yf.download(tickers=long_stocks,period='1d')[['Open','Close']]
short=yf.download(tickers=short_stocks,period='1d')[['Open','Close']]
slippage=0.002
long['Close']=(1-slippage)*long['Close']
short['Close']=(1+slippage)*short['Close']

long['Open']=(1+slippage)*long['Open']
short['Open']=(1-slippage)*short['Open']


long_return = ((long['Close'] - long['Open']) / long['Open']).mean()
short_return = ((-1) * (short['Close'] - short['Open']) / short['Open']).mean()

print(f"Average return for long positions: {5*long_return.mean():.2%}")
print(f"Average return for short positions: {5*short_return.mean():.2%}")

[*********************100%***********************]  500 of 500 completed
              Gap
Stocks           
MMTC.NS -0.049889
Empty DataFrame
Columns: [Gap]
Index: []
['CONCORDBIO.NS', 'TORNTPOWER.NS', 'PATANJALI.NS', 'RAMCOCEM.NS'] ['TIINDIA.NS', 'BHEL.NS', 'ZOMATO.NS', 'CLEAN.NS']
[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  4 of 4 completed
Average return for long positions: 3.14%
Average return for short positions: 5.39%


In [1]:
import yfinance as yf
import pandas as pd
from StockFetcher import get_long_short_universe

stocks_list = pd.read_csv('ind_nifty500list.csv')['Symbol'].to_list()

In [2]:
dfx=yf.download(tickers=stocks_list,period='14d')['Close']

[*********************100%***********************]  500 of 500 completed


In [5]:
dfx.dropna().dropna(axis=1)

,360ONE.NS,3MINDIA.NS,AARTIDRUGS.NS,AARTIIND.NS,AAVAS.NS,ABB.NS,ABBOTINDIA.NS,ABCAPITAL.NS,ABFRL.NS,ACC.NS,...,WELSPUNLIV.NS,WESTLIFE.NS,WHIRLPOOL.NS,WIPRO.NS,YESBANK.NS,ZENSARTECH.NS,ZFCVINDIA.NS,ZOMATO.NS,ZYDUSLIFE.NS,ZYDUSWELL.NS
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-23,619.000000,33527.699219,535.450012,642.400024,1502.150024,4770.549805,25481.400391,163.500000,222.699997,2207.949951,...,142.350006,827.700012,1308.449951,469.899994,24.049999,544.200012,17262.349609,130.100006,729.150024,1582.199951
2024-01-24,617.000000,34470.949219,535.250000,646.000000,1493.699951,4731.250000,25598.599609,165.800003,235.300003,2240.600098,...,135.949997,832.450012,1339.750000,478.049988,24.700001,569.950012,17270.449219,136.199997,752.400024,1602.949951
2024-01-25,630.200012,34048.250000,504.549988,634.099976,1508.250000,4739.299805,25060.750000,165.750000,241.600006,2467.649902,...,137.399994,827.950012,1317.000000,470.000000,24.850000,575.950012,16715.199219,136.149994,736.500000,1601.699951
2024-01-29,634.000000,33724.148438,506.100006,642.349976,1516.250000,4791.549805,25568.050781,168.300003,243.050003,2492.899902,...,149.949997,815.250000,1333.000000,473.000000,24.350000,570.599976,16790.599609,135.550003,748.500000,1595.050049
2024-01-30,628.599976,33655.550781,514.450012,633.549988,1494.300049,4729.750000,25508.650391,168.449997,240.000000,2513.149902,...,152.800003,815.950012,1309.099976,472.500000,23.900000,566.099976,16312.349609,137.199997,755.900024,1601.500000
2024-01-31,620.049988,34408.351562,526.650024,659.099976,1476.550049,4671.600098,25906.949219,171.100006,243.949997,2544.300049,...,163.800003,831.500000,1348.650024,478.149994,24.100000,574.250000,16336.549805,139.550003,760.799988,1614.949951
2024-02-01,617.500000,33803.851562,527.049988,643.400024,1504.849976,4546.049805,25887.000000,166.850006,241.899994,2530.000000,...,163.149994,829.599976,1344.900024,471.350006,23.900000,566.599976,15423.250000,140.550003,759.549988,1599.349976
2024-02-02,604.950012,32868.800781,509.350006,644.700012,1484.050049,4480.450195,28064.300781,180.350006,248.449997,2499.449951,...,156.250000,846.650024,1346.800049,483.149994,23.700001,560.950012,14902.950195,143.800003,761.150024,1598.650024
2024-02-05,615.500000,32380.949219,520.549988,643.799988,1438.500000,4368.200195,28069.500000,178.199997,248.050003,2486.199951,...,152.300003,837.549988,1354.500000,482.350006,22.799999,564.400024,14434.950195,140.250000,760.900024,1589.699951


In [6]:
from Autologin2 import generateToken

In [7]:
from fyers_apiv3 import fyersModel
token= generateToken()
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")


send_login_otp success
generate_totp success
694344
verify_totp_result success
verify_pin_result success
token_result success
validate_authcode success
access_token - eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MDczMTYxMTcsImV4cCI6MTcwNzM1MjIzNywibmJmIjoxNzA3MzE2MTE3LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbHc1T1ZXaEtCOVhaZVdFWFA2Yk5zbkdScDFfQk1qcjFnQUF1RGdZMVN6dHVoTXJaUVdQUW0xVHpMN2JPOXladE5mQk9LSV8wdnJkU2dmTVRaU3pnZW9pMGRseGpyUWc5dGQ1UWlTNGZaV3hvbnlxaz0iLCJkaXNwbGF5X25hbWUiOiJLSVNIT1JCSEFJIERIQU5KSUJIQUkgVEhBS0tBUiIsIm9tcyI6IksxIiwiaHNtX2tleSI6IjhhMTUyMDYzMTBlZWMyODFiYjVkZDEwMTk1YzRiNzlmNmRkNDI1OTM4YjgxMjk4MzRhY2JiMzM3IiwiZnlfaWQiOiJESzAxNzMxIiwiYXBwVHlwZSI6MTAwLCJwb2FfZmxhZyI6Ik4ifQ.Hnc4VbX5Bwa_6jZ9LpANKLyOQH_57j9mVfgLqxsgORM


NameError: name 'client_id' is not defined

In [24]:
import os
directory_capital = r"C:\Users\Sohum\Desktop\Long Short Deployment"

daily_profit = fyers.positions()["overall"]['pl_realized']
result = daily_profit - 0.1*abs(daily_profit) + 160000
if not os.path.exists(directory_capital):
    os.makedirs(directory_capital)

file_path = os.path.join(directory_capital, "InitialCapital.txt")
with open(file_path, "w") as file:
    file.write(str(result))

In [87]:
df = pd.DataFrame(fyers.orderbook()['orderBook'])
df = df[['id', 'qty', 'symbol', 'orderDateTime', 'side', 'tradedPrice', 'orderNumStatus', 'slNo']]

# Your project folder path
project_folder = r"C:\Users\Sohum\Desktop\Long Short Deployment"

csv_file_path = os.path.join(project_folder, "tradebook.csv")

# Append the DataFrame to the CSV file
df.to_csv(csv_file_path, mode='a', header=not os.path.exists(csv_file_path), index=False)

print("DataFrame appended to CSV:", csv_file_path)

DataFrame appended to CSV: C:\Users\Sohum\Desktop\Long Short Deployment\tradebook.csv


In [3]:
import yfinance  as yf 
import pandas as pd
df=yf.download(tickers=stocks_list,period='7d')['Close']


[*********************100%***********************]  501 of 501 completed


In [ ]:
import pandas as pd
from StockFetcher import get_long_short_universe
stocks_list = pd.read_csv('ind_nifty500list.csv')['Symbol'].to_list()

long_stocks,short_stocks=get_long_short_universe(stocks_list,2)
long_stocks,short_stocks

[******                12%                       ]  59 of 501 completed

KeyboardInterrupt: 

[*********************100%***********************]  499 of 501 completed

In [30]:
file_path = os.path.join(directory_capital, "InitialCapital.txt")
with open(file_path, "r") as file:
            retrieved_result = str(file.read())

In [38]:
int(float(retrieved_result))

161103

In [6]:
from Autologin import generateToken
from fyers_apiv3 import fyersModel

client_id = "CDAAGPIZ8E-100"
access_token = generateToken()
while not access_token:
    print("Failed to obtain access token. Retrying...")
    access_token = generateToken()
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")


send_login_otp success
generate_totp success
162816
verify_totp_result success
verify_pin_result success
token_result success
validate_authcode success
access_token - eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MDY0NjY1NjcsImV4cCI6MTcwNjQ4ODIyNywibmJmIjoxNzA2NDY2NTY3LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbHRwMEg4WnlMZk1neW5vazY4T21lSlVCYlJjU0t4NmJsSS1aSWlWSXJxc0M2WUM5WDZuZC0yMFEyZmx0VVdxZzdoMEJPMlZKazNYS2xpODV0NXNhYlpsSnZjYVJ6UTBfbmpWdWNZT0tRUVFKQVJNOD0iLCJkaXNwbGF5X25hbWUiOiJTT0hVTSBNVUxFWSIsIm9tcyI6IksxIiwiaHNtX2tleSI6ImYxYzA3NWQ0Y2Q2ZmQ4MjM0ZTM4MDdmNWI2OTEzYTMzOGRiNDQ1ODU1ZGJhMWZmM2Q0YTc1NTIyIiwiZnlfaWQiOiJYUzU1MDgzIiwiYXBwVHlwZSI6MTAwLCJwb2FfZmxhZyI6Ik4ifQ.DzUTJoRWD8t9WiSrEcq9rp40pwo4tW0AmWCJpzOQ9U4


In [8]:
fyers.funds()

{'code': 200,
 'message': '',
 's': 'ok',
 'fund_limit': [{'id': 1,
   'title': 'Total Balance',
   'equityAmount': 11102.7,
   'commodityAmount': 0},
  {'id': 2,
   'title': 'Utilized Amount',
   'equityAmount': 0,
   'commodityAmount': 0},
  {'id': 3,
   'title': 'Clear Balance',
   'equityAmount': 11102.7,
   'commodityAmount': 0},
  {'id': 4,
   'title': 'Realized Profit and Loss',
   'equityAmount': 0,
   'commodityAmount': 0},
  {'id': 5, 'title': 'Collaterals', 'equityAmount': 0, 'commodityAmount': 0},
  {'id': 6, 'title': 'Fund Transfer', 'equityAmount': 0, 'commodityAmount': 0},
  {'id': 7, 'title': 'Receivables', 'equityAmount': 0, 'commodityAmount': 0},
  {'id': 8, 'title': 'Adhoc Limit', 'equityAmount': 0, 'commodityAmount': 0},
  {'id': 9,
   'title': 'Limit at start of the day',
   'equityAmount': 11102.7,
   'commodityAmount': 0},
  {'id': 10,
   'title': 'Available Balance',
   'equityAmount': 11102.7,
   'commodityAmount': 0}]}

In [3]:
def create_order(symbol, side,capital):

    data = {
        "symbol":symbol,
        "resolution":"D",
        "date_format":"0",
        "range_from":date_to_epoch(),
        "range_to":date_to_epoch(),
        "cont_flag":"1"
    }
    response = fyers.history(data=data)
    openprice=float(pd.DataFrame(response['candles'],columns=['Date','Open','High','Low','Close','Volume'])['Open'])
    quantity= int(capital/openprice)
    order_dict = {
        "symbol": symbol,
        "qty": quantity,
        "type": 2,
        "side": side,
        "productType": "INTRADAY",
        "limitPrice": 0,
        "stopPrice": 0,
        "validity": "DAY",
        "disclosedQty": 0,
        "offlineOrder": False,
    }
    return order_dict

In [3]:
import yfinance as yf
import pandas as pd
stocks_list = pd.read_csv('ind_nifty500list.csv')['Symbol'].to_list()

stocks=yf.download(tickers=stocks_list[0:3],period='10d')

[*********************100%***********************]  3 of 3 completed


In [4]:
stocks.dropna(axis=1, how='all')

Adj Close                                  Close                \
             360ONE.NS    3MINDIA.NS       ABB.NS   360ONE.NS    3MINDIA.NS   
Date                                                                          
2024-01-23  614.579956  33527.699219  4770.549805  619.000000  33527.699219   
2024-01-24  612.594238  34470.949219  4731.250000  617.000000  34470.949219   
2024-01-25  625.700012  34048.250000  4739.299805  630.200012  34048.250000   
2024-01-29  634.000000  33724.148438  4791.549805  634.000000  33724.148438   
2024-01-30  628.599976  33655.550781  4729.750000  628.599976  33655.550781   
2024-01-31  620.049988  34408.351562  4671.600098  620.049988  34408.351562   
2024-02-01  617.500000  33803.851562  4546.049805  617.500000  33803.851562   
2024-02-02  604.950012  32868.800781  4480.450195  604.950012  32868.800781   
2024-02-05  615.500000  32380.949219  4368.200195  615.500000  32380.949219   
2024-02-06  628.500000  33123.250000  4403.000000  628.500000  33123.250000   
2024-02-07  628.500000  33123.250000  4403.000000  628.500000  33123.250000   

                               High                                    Low  \
                 ABB.NS   360ONE.NS    3MINDIA.NS       ABB.NS   360ONE.NS   
Date                                                                         
2024-01-23  4770.549805  644.900024  34850.000000  4897.000000  610.000000   
2024-01-24  4731.250000  627.000000  34649.898438  4814.899902  591.500000   
2024-01-25  4739.299805  637.400024  34756.101562  4794.950195  614.950012   
2024-01-29  4791.549805  643.549988  34259.949219  4807.149902  626.000000   
2024-01-30  4729.750000  642.000000  34059.949219  4860.000000  618.849976   
2024-01-31  4671.600098  640.650024  34659.050781  4762.750000  616.150024   
2024-02-01  4546.049805  624.799988  34500.000000  4698.000000  609.799988   
2024-02-02  4480.450195  620.599976  34039.000000  4594.950195  601.099976   
2024-02-05  4368.200195  621.299988  32975.000000  4538.500000  597.250000   
2024-02-06  4403.000000  639.000000  33769.000000  4423.600098  615.000000   
2024-02-07  4403.000000  639.900024  33500.000000  4434.549805  639.900024   

                                             Open                             \
              3MINDIA.NS       ABB.NS   360ONE.NS    3MINDIA.NS       ABB.NS   
Date                                                                           
2024-01-23  33305.000000  4735.899902  644.000000  34850.000000  4750.950195   
2024-01-24  33605.250000  4613.799805  620.000000  34188.000000  4764.950195   
2024-01-25  33950.000000  4665.000000  617.000000  34166.000000  4768.000000   
2024-01-29  33456.000000  4715.399902  633.000000  34259.949219  4751.000000   
2024-01-30  33569.000000  4715.950195  639.950012  33860.000000  4825.000000   
2024-01-31  33655.550781  4631.000000  630.349976  33655.550781  4749.000000   
2024-02-01  33700.000000  4340.299805  621.000000  34450.851562  4692.000000   
2024-02-02  32800.000000  4463.000000  620.000000  33998.898438  4546.049805   
2024-02-05  32201.000000  4343.750000  610.000000  32975.000000  4500.000000   
2024-02-06  32411.250000  4362.049805  623.450012  32580.949219  4378.000000   
2024-02-07  33500.000000  4434.549805  639.900024  33500.000000  4434.549805   

              Volume                     
           360ONE.NS 3MINDIA.NS  ABB.NS  
Date                                     
2024-01-23    448657       4424  348426  
2024-01-24    332842       4769  293735  
2024-01-25    244105       2771  114581  
2024-01-29    899275       2703   87854  
2024-01-30    170252       1639  152802  
2024-01-31    347921       4845  292837  
2024-02-01    364500       2999  825760  
2024-02-02    290046       5414  315504  
2024-02-05    381313       6870  290961  
2024-02-06   1281972       6875  177927  
2024-02-07   1281982       6876  177928

In [7]:
from Autologin2 import generateToken
client_id = "R4NUGDSC7E-100"
access_token = generateToken()

send_login_otp success
generate_totp success
372833
verify_totp_result success
verify_pin_result success
token_result success
validate_authcode success
access_token - eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE3MDcyNzcyNDksImV4cCI6MTcwNzM1MjI0OSwibmJmIjoxNzA3Mjc3MjQ5LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbHd2dkJYOWRoTngwSi1pek1NTnIxTlgtZmllQmE0QXNrWDZfbzk0MzZaUjF5RGdKRDlfc1d5WDBJT0lGZDhncXo3Zi0tQjkzT3hfeFNlMDYwa3VsZkNTcVNyTm9nLURJRGZuSGQ2c1hvNVcwS3dEcz0iLCJkaXNwbGF5X25hbWUiOiJLSVNIT1JCSEFJIERIQU5KSUJIQUkgVEhBS0tBUiIsIm9tcyI6IksxIiwiaHNtX2tleSI6IjhhMTUyMDYzMTBlZWMyODFiYjVkZDEwMTk1YzRiNzlmNmRkNDI1OTM4YjgxMjk4MzRhY2JiMzM3IiwiZnlfaWQiOiJESzAxNzMxIiwiYXBwVHlwZSI6MTAwLCJwb2FfZmxhZyI6Ik4ifQ.coca5zbK61_nMJyV0KwiLd_Uak0za6NJ6n4QSKsP46c


In [8]:
import json
import requests
import pyotp
from urllib import parse
import sys
import hashlib
def generate_totp(secret):
    try:
        generated_totp = pyotp.TOTP(secret).now()
        return [generated_totp]
    
    except Exception as e:
        return [e]
    




In [10]:
generate_totp('6ASMDEPJQAKCBQHPM66KH6BZE5Q4323L')

['877321']

In [10]:
import pandas as pd
import datetime


def getIntradayBrokerage(buyAt,sellAt,quantity):
    brokerage = min((buyAt * quantity * 0.0003),20) + min((sellAt * quantity * 0.0003),20)
    turnover = (buyAt+sellAt)*quantity
    stt = ((sellAt * quantity) * 0.00025)
    sebi = turnover*0.000001
    ex_txn_charge = (0.0000325*turnover) + (0.000001 * turnover)
    stax = (0.18 * (brokerage + ex_txn_charge + sebi))
    stamp = (buyAt*quantity)*0.00003
    return brokerage + stt + ex_txn_charge + stax + sebi + stamp








Trades=pd.read_csv('tradebook2.csv')

Trades['orderDateTime']=pd.to_datetime(Trades['orderDateTime'])
Trades=Trades.loc[Trades['orderDateTime'].dt.date == datetime.datetime.today().date()].loc[Trades['filledQty']!=0]
Trades=Trades.loc[Trades['filledQty']!=0]
Exits,Entries=Trades.loc[Trades['orderTag']=='2:Exit'],Trades.loc[Trades['orderTag']!='2:Exit']


lst = []
for i in range(len(Entries['symbol'])):
    stock = Entries['symbol'].iloc[i]
    
    # Check if the symbol exists in both Entry and Exit DataFrames
    if stock in Entries['symbol'].values and stock in Exits['symbol'].values:
        stock_entry = Entries.loc[Entries['symbol'] == stock]
        stock_exit = Exits.loc[Exits['symbol'] == stock]

        lst.append((stock,
                    stock_entry['tradedPrice'].iloc[0],
                    stock_exit['tradedPrice'].iloc[0],
                    stock_entry['side'].iloc[0],
                    stock_entry['filledQty'].iloc[0]))
    else:
        continue


BookForTheDay=pd.DataFrame(lst,columns=['Stock','Entry','Exit','Side','Quantity'])
BookForTheDay['Brokerage']=0
BookForTheDay['Brokerage']=BookForTheDay.apply(lambda row: getIntradayBrokerage(row['Entry'], row['Exit'], row['Quantity']), axis=1)

BookForTheDay['Brokerage'].sum()


3960.5712514992

In [17]:
-BookForTheDay.drop_duplicates('Stock')['Brokerage'].sum()

-660.0952085832